# Detecting neuron selectivity with INTENSE

Which neurons encode which behavioral variables -- and how do you tell
real selectivity from chance?  [**DRIADA**](https://driada.readthedocs.io)
answers this with **INTENSE**, an information-theoretic significance
testing pipeline.  This notebook walks through the method from first
principles to a full production run.

| Step | Notebook | What it does |
|---|---|---|
| **Overview** | [00 -- DRIADA overview](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/00_driada_overview.ipynb) | Core data structures, quick tour of INTENSE, DR, networks |
| Neuron analysis | [01 -- Neuron analysis](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/01_data_loading_and_neurons.ipynb) | Spike reconstruction, kinetics optimization, quality metrics, surrogates |
| **Single-neuron selectivity** | **02 -- this notebook** | Detect which neurons encode which behavioral variables |
| Population geometry | [03 -- Dimensionality reduction](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/03_population_geometry_dr.ipynb) | Extract low-dimensional manifolds from population activity |
| Network analysis | [04 -- Networks](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/04_network_analysis.ipynb) | Build and analyze cell-cell interaction graphs |
| Putting it together | [05 -- Advanced](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/05_advanced_capabilities.ipynb) | Combine INTENSE + DR, leave-one-out importance, RSA, RNN analysis |

**What you will learn:**

1. **Information theory fundamentals** -- mutual information estimation
   (GCMI vs KSG), similarity metrics, time-delayed MI, conditional MI,
   and interaction information.
2. **Basic INTENSE workflow** -- generate a synthetic population, run
   two-stage significance testing, and extract results.
3. **Complete pipeline with ground truth** -- feature-feature correlations,
   all feature types, Holm correction, disentanglement, delay optimization,
   and validation against known selectivity.

In [ ]:
# TODO: revert to '!pip install -q driada' after v1.0.0 PyPI release
!pip install -q git+https://github.com/iabs-neuro/driada.git@main
%matplotlib inline

import os
import time
import tempfile

import numpy as np
import matplotlib.pyplot as plt

import driada
from driada.information import (
    TimeSeries,
    get_mi,
    get_sim,
    get_tdmi,
    conditional_mi,
    interaction_information,
)
from driada.experiment.synthetic import generate_tuned_selectivity_exp
from driada.intense import compute_feat_feat_significance
from driada.intense.io import save_results, load_results

## 1. Information theory fundamentals

Before running INTENSE, understand the building blocks: mutual information
estimation, similarity metrics, temporal lags, and conditional dependencies.

### Creating TimeSeries

Wrap numpy arrays as [`TimeSeries`](https://driada.readthedocs.io/en/latest/api/information/core.html#driada.information.info_base.TimeSeries) with type hints (`linear`, `categorical`,
`circular`). The type determines which MI estimator DRIADA uses:

| Pair type | GCMI (fast, copula-based) | KSG (accurate, nearest-neighbor) |
|---|---|---|
| continuous–continuous | [`mi_gg`](https://driada.readthedocs.io/en/latest/api/information/mutual_information.html#mi-gg) | [`nonparam_mi_cc`](https://driada.readthedocs.io/en/latest/api/information/mutual_information.html#nonparam-mi-cc) |
| continuous–discrete | [`mi_model_gd`](https://driada.readthedocs.io/en/latest/api/information/mutual_information.html#mi-model-gd) | [`nonparam_mi_cd`](https://driada.readthedocs.io/en/latest/api/information/mutual_information.html#nonparam-mi-cd) |
| discrete–discrete | exact MI from joint distribution | — |

In [ ]:
rng = np.random.default_rng(42)
n = 5000

print("[1] Creating TimeSeries objects")
print("-" * 40)

continuous = rng.normal(size=n)
ts_cont = TimeSeries(continuous, ts_type="linear", name="continuous")
print(f"  Continuous: type={ts_cont.type_info}, len={len(ts_cont.data)}")

discrete = rng.choice([0, 1, 2], size=n, p=[0.5, 0.3, 0.2])
ts_disc = TimeSeries(discrete, ts_type="categorical", name="discrete")
print(f"  Discrete:   type={ts_disc.type_info}, len={len(ts_disc.data)}")

circular = rng.uniform(0, 2 * np.pi, size=n)
ts_circ = TimeSeries(circular, ts_type="circular", name="circular")
print(f"  Circular:   type={ts_circ.type_info}, len={len(ts_circ.data)}")

### Pairwise MI: GCMI vs KSG

[`get_mi`](https://driada.readthedocs.io/en/latest/api/information/core.html#driada.information.info_base.get_mi)`()` estimates mutual information between two `TimeSeries`.
[**GCMI**](https://driada.readthedocs.io/en/latest/api/information/mutual_information.html#gaussian-copula-mi-gcmi)
(Gaussian Copula MI) is a fast, copula-based estimator but can underestimate
non-monotonic dependency.
[**KSG**](https://driada.readthedocs.io/en/latest/api/information/mutual_information.html#ksg-estimators)
(Kraskov-Stögbauer-Grassberger) is nearest-neighbor-based and captures
arbitrary dependency but is slower.

In [ ]:
print("[2] Pairwise mutual information (get_mi)")
print("-" * 40)

x = rng.normal(size=n)
noise = rng.normal(size=n)
y_corr = x + 0.5 * noise          # correlated with x
y_indep = rng.normal(size=n)       # independent of x

ts_x = TimeSeries(x)
ts_y_corr = TimeSeries(y_corr)
ts_y_indep = TimeSeries(y_indep)

mi_corr = get_mi(ts_x, ts_y_corr)
mi_indep = get_mi(ts_x, ts_y_indep)
print(f"  MI(X, Y_correlated)  = {mi_corr:.4f} bits")
print(f"  MI(X, Y_independent) = {mi_indep:.4f} bits")
print(f"  Correlated MI >> independent MI: {mi_corr > 5 * mi_indep}")

mi_gcmi = get_mi(ts_x, ts_y_corr, estimator="gcmi")
mi_ksg = get_mi(ts_x, ts_y_corr, estimator="ksg")
print(f"\n  Monotonic relationship (y = x + noise):")
print(f"    GCMI: {mi_gcmi:.4f} bits")
print(f"    KSG:  {mi_ksg:.4f} bits")
print(f"    (agree because relationship is monotonic)")

# Spearman rho ~ 0 due to exact symmetry, so GCMI ~ 0.
x_sym = rng.uniform(-3, 3, size=n)
y_quad = x_sym ** 2 + 0.3 * rng.normal(size=n)
ts_x_sym = TimeSeries(x_sym)
ts_y_quad = TimeSeries(y_quad)
mi_gcmi_q = get_mi(ts_x_sym, ts_y_quad, estimator="gcmi")
mi_ksg_q = get_mi(ts_x_sym, ts_y_quad, estimator="ksg")
print(f"\n  Non-monotonic relationship (y = x^2 + noise):")
print(f"    GCMI: {mi_gcmi_q:.4f} bits  (underestimates symmetric dependency)")
print(f"    KSG:  {mi_ksg_q:.4f} bits  (captures it)")
print(f"    KSG >> GCMI: {mi_ksg_q > 3 * mi_gcmi_q}")

### Similarity metrics

[`get_sim`](https://driada.readthedocs.io/en/latest/api/information/mutual_information.html#driada.information.get_sim)`()` wraps MI, Pearson r, and Spearman rho in a unified interface.
Available metrics include `mi`, `pearsonr`, `spearmanr`, `kendalltau`,
`fast_pearsonr`, `av` (activity ratio for binary-gated signals), and any
scipy.stats correlation function by name.

In [ ]:
print("[3] Similarity metrics comparison (get_sim)")
print("-" * 40)

metrics = ["mi", "pearsonr", "spearmanr"]
for metric in metrics:
    val = get_sim(ts_x, ts_y_corr, metric=metric)
    print(f"  {metric:12s}(X, Y_corr) = {val:.4f}")

### Time-delayed MI

[`get_tdmi`](https://driada.readthedocs.io/en/latest/api/information/core.html#driada.information.info_base.get_tdmi)`()` sweeps temporal lags to find the shift that maximizes MI.
This is useful for detecting delayed neural responses to behavior.

In [ ]:
# Noisy sine wave — TDMI reveals the underlying periodicity
period_samples = 30
t_demo = np.arange(n)
clean_sine = np.sin(2 * np.pi * t_demo / period_samples)
signal = clean_sine + 0.4 * rng.normal(size=n)

max_shift = 60  # lag window in samples (= 3 sec at 20 fps)
tdmi_values = np.array(get_tdmi(signal, max_shift=max_shift))
lags = np.arange(1, max_shift)
best_lag = lags[np.argmax(tdmi_values)]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 4),
                                gridspec_kw={'height_ratios': [1, 1.3]})

show_n = 4 * period_samples
ax1.plot(t_demo[:show_n], signal[:show_n], alpha=0.6, label='noisy')
ax1.plot(t_demo[:show_n], clean_sine[:show_n], 'k--', alpha=0.4, label='clean')
ax1.set_ylabel('Amplitude')
ax1.set_title(f'Noisy sine wave (period = {period_samples} samples)')
ax1.legend(fontsize=8)

ax2.plot(lags, tdmi_values, 'k-')
ax2.axvline(best_lag, color='red', ls='--',
            label=f'Peak lag = {best_lag} samples')
ax2.set_xlabel('Lag (samples)')
ax2.set_ylabel('MI (bits)')
ax2.set_title('Time-delayed MI')
ax2.legend(fontsize=8)

plt.tight_layout()
plt.show()

print(f'Signal period: {period_samples} samples')
print(f'TDMI peak:     {best_lag} samples')

### Conditional MI and interaction information

**Conditional MI** ([`conditional_mi`](https://driada.readthedocs.io/en/latest/api/information/core.html#driada.information.info_base.conditional_mi)) `I(X;Y|Z)` removes shared variance with Z.
**Interaction information** ([`interaction_information`](https://driada.readthedocs.io/en/latest/api/information/core.html#driada.information.info_base.interaction_information)) distinguishes synergy (>0) from
redundancy (<0).

In [ ]:
print("[5] Conditional MI: I(X;Y|Z)")
print("-" * 40)

z = rng.normal(size=n)
x_from_z = z + 0.3 * rng.normal(size=n)
y_from_z = z + 0.3 * rng.normal(size=n)

ts_xz = TimeSeries(x_from_z)
ts_yz = TimeSeries(y_from_z)
ts_z = TimeSeries(z)

mi_xy = get_mi(ts_xz, ts_yz)
cmi_xy_z = conditional_mi(ts_xz, ts_yz, ts_z)

print(f"  I(X;Y)   = {mi_xy:.4f} bits  (shared via Z)")
print(f"  I(X;Y|Z) = {cmi_xy_z:.4f} bits  (residual after conditioning)")
print(f"  Conditioning reduces MI: {cmi_xy_z < mi_xy * 0.5}")

In [ ]:
print("[6] Interaction information: synergy vs redundancy")
print("-" * 40)

# Redundancy: Y and Z provide overlapping info about X
x_r = rng.normal(size=n)
y_r = TimeSeries(x_r + 0.2 * rng.normal(size=n))
z_r = TimeSeries(x_r + 0.2 * rng.normal(size=n))
ts_xr = TimeSeries(x_r)

ii_redund = interaction_information(ts_xr, y_r, z_r)
print(f"  Redundancy example: II = {ii_redund:.4f} (expected < 0)")

# Synergy: XOR-like relationship
a = rng.choice([0, 1], size=n).astype(float)
b = rng.choice([0, 1], size=n).astype(float)
xor_signal = (a + b + 0.1 * rng.normal(size=n))

ts_xor = TimeSeries(xor_signal)
ts_a = TimeSeries(a, ts_type="binary")
ts_b = TimeSeries(b, ts_type="binary")

ii_synergy = interaction_information(ts_xor, ts_a, ts_b)
print(f"  Synergy example:    II = {ii_synergy:.4f} (expected > 0)")
print(f"  Redundancy is negative: {ii_redund < 0}")

## 2. Basic INTENSE workflow

These information-theoretic tools are the building blocks. DRIADA's INTENSE
pipeline combines them into a two-stage significance test that scales to
hundreds of neurons and features.

The minimal pipeline: [`generate_tuned_selectivity_exp`](https://driada.readthedocs.io/en/latest/api/experiment/synthetic.html#driada.experiment.synthetic.generators.generate_tuned_selectivity_exp) creates a synthetic
population with known selectivity, [`compute_cell_feat_significance`](https://driada.readthedocs.io/en/latest/api/intense/pipelines.html#driada.intense.pipelines.compute_cell_feat_significance) runs
two-stage significance testing, and [`plot_neuron_feature_pair`](https://driada.readthedocs.io/en/latest/api/intense/visual.html#driada.intense.visual.plot_neuron_feature_pair) visualizes
individual neuron-feature relationships.

For loading your own experimental data into an `Experiment` object, see
[Notebook 00](https://colab.research.google.com/github/iabs-neuro/driada/blob/main/notebooks/00_driada_overview.ipynb).
The workflow below is identical for real recordings.

In [ ]:
print("Generating synthetic experiment...")

population = [
    {"name": "speed_cells", "count": 3, "features": ["speed"]},
    {"name": "event_cells", "count": 3, "features": ["event_0"]},
    {"name": "nonselective", "count": 4, "features": []},
]

exp = generate_tuned_selectivity_exp(
    population=population,
    duration=600,
    fps=20,
    seed=47,
    n_discrete_features=1,
    verbose=True,
)

In [ ]:
print("Running INTENSE analysis...")

stats, significance, info, results = driada.compute_cell_feat_significance(
    exp,
    mode="two_stage",
    n_shuffles_stage1=100,   # fast screening
    n_shuffles_stage2=10000,  # precise p-values via gamma fit
    pval_thr=0.001,
    multicomp_correction=None,
    ds=5,
    verbose=True,
)

In [ ]:
significant_neurons = exp.get_significant_neurons()
total_pairs = sum(len(features) for features in significant_neurons.values())

print(f"Significant neurons: {len(significant_neurons)}/{exp.n_cells}")
print(f"Total neuron-feature pairs: {total_pairs}")

for cell_id, features in significant_neurons.items():
    for feat_name in features:
        pair_stats = exp.get_neuron_feature_pair_stats(cell_id, feat_name)
        mi = pair_stats.get('me', 0)
        pval = pair_stats.get('pval', 1.0)
        print(f"  Neuron {cell_id} -> {feat_name}: MI={mi:.4f}, p={pval:.2e}")

In [ ]:
# Visualize an event neuron — shaded regions show when the event is active
event_neurons = [cid for cid, feats in significant_neurons.items()
                 if 'event_0' in feats]
if event_neurons:
    plot_id = event_neurons[0]
    fig = driada.intense.plot_neuron_feature_pair(exp, plot_id, 'event_0')
    plt.show()

## 3. Complete pipeline with ground truth validation

The basic workflow above used a small population and default settings. Next,
we set up a realistic pipeline with ground truth to evaluate detection
accuracy.

Production workflow: feature-feature correlation analysis, all feature types
(circular, spatial, linear, discrete), Holm correction, disentanglement,
delay optimization, and ground truth validation.

In [ ]:
# Population configuration - defines neuron groups and their selectivity
POPULATION = [
    {"name": "hd_cells", "count": 4, "features": ["head_direction"]},
    {"name": "place_cells", "count": 4, "features": ["position_2d"]},
    {"name": "speed_cells", "count": 4, "features": ["speed"]},
    {"name": "event_cells", "count": 4, "features": ["event_0"]},
    {"name": "mixed_cells", "count": 4, "features": ["head_direction", "event_0"]},
    {"name": "nonselective", "count": 4, "features": []},
]

# Analysis parameters
CONFIG = {
    # Recording parameters
    "duration": 900,        # seconds
    "fps": 20,              # sampling rate
    "seed": 42,
    # Tuning parameters
    "kappa": 4.0,           # von Mises concentration (HD cells)
    # Calcium dynamics
    "baseline_rate": 0.02,  # baseline firing rate
    "peak_rate": 2.0,       # peak response
    "decay_time": 1.5,      # calcium decay time
    "calcium_noise": 0.01,  # noise level
    # Discrete event parameters
    "n_discrete_features": 2,
    "event_active_fraction": 0.08,  # ~8% active time per event
    "event_avg_duration": 0.8,      # seconds
    # INTENSE analysis parameters
    "n_shuffles_stage1": 100,   # stage 1 screening shuffles
    "n_shuffles_stage2": 10000,  # stage 2 confirmation (FFT makes this fast)
    "pval_thr": 0.05,           # p-value threshold after correction
    "multicomp_correction": "holm",  # multiple comparison correction
}

# Custom tuning defaults based on config
tuning_defaults = {
    "head_direction": {"kappa": CONFIG["kappa"]},
}

exp3 = generate_tuned_selectivity_exp(
    population=POPULATION,
    tuning_defaults=tuning_defaults,
    duration=CONFIG["duration"],
    fps=CONFIG["fps"],
    baseline_rate=CONFIG["baseline_rate"],
    peak_rate=CONFIG["peak_rate"],
    decay_time=CONFIG["decay_time"],
    calcium_noise=CONFIG["calcium_noise"],
    n_discrete_features=CONFIG["n_discrete_features"],
    event_active_fraction=CONFIG["event_active_fraction"],
    event_avg_duration=CONFIG["event_avg_duration"],
    seed=CONFIG["seed"],
    verbose=True,
)
ground_truth = exp3.ground_truth

# Add a derived feature: speed with measurement noise (correlated by construction)
speed_data = exp3.dynamic_features['speed'].data
rng = np.random.RandomState(CONFIG['seed'])
speed_noisy = speed_data + rng.normal(0, 0.3 * np.std(speed_data), len(speed_data))
exp3.add_feature('speed_noisy', speed_noisy, ts_type='linear')
from scipy.stats import pearsonr
r, _ = pearsonr(speed_data, speed_noisy)
print(f'  Added speed_noisy (speed + 30% Gaussian noise, r={r:.3f})')

### Feature-feature correlations

Before analyzing neurons, check which behavioral variables are themselves
correlated. [`compute_feat_feat_significance`](https://driada.readthedocs.io/en/latest/api/intense/pipelines.html#driada.intense.pipelines.compute_feat_feat_significance)
tests all feature pairs with FFT-based circular shuffles. For example,
the noisy speed is derived from speed by adding measurement noise, so they
are correlated by construction. Any significant correlations here will inform
the disentanglement step later.

In [ ]:
feat_bunch = [
    feat_name for feat_name in exp3.dynamic_features.keys()
    if feat_name not in ['x', 'y', 'head_direction']
]
print(f'Features to test: {feat_bunch}')

sim_mat, sig_mat, pval_mat, feature_names_ff, info_ff = compute_feat_feat_significance(
    exp3,
    feat_bunch=feat_bunch,
    n_shuffles_stage1=100,
    n_shuffles_stage2=1000,
    pval_thr=0.01,
    verbose=True,
)

In [ ]:
display_names = []
for name in feature_names_ff:
    if isinstance(name, (list, tuple)):
        display_names.append(', '.join(str(n) for n in name))
    else:
        display_names.append(str(name))

n_ff = len(feature_names_ff)

print('Significant pairs:')
n_sig = 0
for i in range(n_ff):
    for j in range(i + 1, n_ff):
        if sig_mat[i, j]:
            n_sig += 1
            print(
                f'  {display_names[i]:20s} <-> {display_names[j]:20s}  '
                f'MI={sim_mat[i, j]:.4f}  p={pval_mat[i, j]:.2e}'
            )
if n_sig == 0:
    print('  (none)')

fig, ax = plt.subplots(figsize=(8, 7))
plot_mat = sim_mat.copy().astype(float)
np.fill_diagonal(plot_mat, np.nan)

im = ax.imshow(plot_mat, cmap='Blues', aspect='equal')
cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Mutual information (bits)')

for i in range(n_ff):
    for j in range(n_ff):
        if i != j and sig_mat[i, j]:
            ax.text(j, i, '*', ha='center', va='center',
                    fontsize=14, fontweight='bold', color='red')

for i in range(n_ff):
    ax.add_patch(plt.Rectangle((i - 0.5, i - 0.5), 1, 1,
                               fill=True, facecolor='0.85', edgecolor='none'))

ax.set_xticks(range(n_ff))
ax.set_xticklabels(display_names, rotation=45, ha='right', fontsize=8)
ax.set_yticks(range(n_ff))
ax.set_yticklabels(display_names, fontsize=8)
ax.set_title('Feature-feature MI (* = significant)')

plt.tight_layout()
plt.show()

### Running INTENSE with disentanglement

Some features are correlated (e.g. speed and noisy speed above). Before
interpreting neuron selectivity, we need to account for these redundancies.
Disentanglement uses conditional MI to separate genuine from inherited
selectivity.

With the feature correlation structure in mind, run the full pipeline
with `with_disentanglement=True` to resolve redundant multi-feature
detections.

In [ ]:
print('Running INTENSE analysis...')
start_time = time.time()

stats3, significance3, info3, results3, disent_results3 = driada.compute_cell_feat_significance(
    exp3,
    feat_bunch=feat_bunch,
    mode='two_stage',
    n_shuffles_stage1=CONFIG['n_shuffles_stage1'],
    n_shuffles_stage2=CONFIG['n_shuffles_stage2'],
    find_optimal_delays=True,
    ds=5,
    pval_thr=CONFIG['pval_thr'],
    multicomp_correction=CONFIG['multicomp_correction'],
    with_disentanglement=True,
    verbose=True,
)

analysis_time = time.time() - start_time
significant_neurons3 = exp3.get_significant_neurons()

total_pairs = sum(len(features) for features in significant_neurons3.values())
print(f"\n  Completed in {analysis_time:.1f} seconds")
print(f"  Significant neurons: {len(significant_neurons3)}/{exp3.n_cells}")
print(f"  Total significant pairs: {total_pairs}")

### Ground truth validation

Compare detections to known selectivity. `validate_against_ground_truth`
computes sensitivity, precision, and F1 per neuron type.

In [ ]:
metrics = results3.validate_against_ground_truth(ground_truth, verbose=True)

### Disentanglement: before vs after

The feature-feature analysis above showed which features correlate.
Disentanglement leverages that: for neurons detected with multiple features,
it tests `I(neuron; F1 | F2) > 0` to decide whether each detection is
truly independent or redundant.

In [ ]:
if disent_results3 is not None:
    summary = disent_results3.get("summary", {})
    per_neuron_disent = disent_results3.get("per_neuron_disent", {})

    if "overall_stats" in summary:
        stats_d = summary["overall_stats"]
        print(f"\n  Neuron-feature pairs analyzed: {stats_d.get('total_neuron_pairs', 0)}")
        print(f"  Redundancy rate: {stats_d.get('redundancy_rate', 0):.1f}%")
        print(f"  True mixed selectivity rate: {stats_d.get('true_mixed_selectivity_rate', 0):.1f}%")

    # Build corrected significant_neurons using final_sels
    corrected = {}
    n_removed = 0
    for neuron_id, features in significant_neurons3.items():
        if neuron_id in per_neuron_disent:
            final = per_neuron_disent[neuron_id].get("final_sels", features)
            n_removed += len(features) - len(final)
            if final:
                corrected[neuron_id] = final
        else:
            corrected[neuron_id] = features

    # Compute corrected metrics against ground truth
    expected_pairs = set(ground_truth["expected_pairs"])
    tp, fp, fn = 0, 0, 0
    for neuron_id, features in corrected.items():
        for feat_name in features:
            if (neuron_id, feat_name) in expected_pairs:
                tp += 1
            else:
                fp += 1
    fn = len(expected_pairs) - tp

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * precision * sensitivity / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

    # Show before/after
    print(f"\n  Pairs removed by disentanglement: {n_removed}")
    print(f"\n  {'Metric':<15} {'Before':<12} {'After'}")
    print(f"  {'-'*40}")
    print(f"  {'Sensitivity':<15} {metrics['sensitivity']:>10.1%}   {sensitivity:>10.1%}")
    print(f"  {'Precision':<15} {metrics['precision']:>10.1%}   {precision:>10.1%}")
    print(f"  {'F1 Score':<15} {metrics['f1']:>10.1%}   {f1:>10.1%}")
    print(f"  {'False Pos':<15} {metrics['false_positives']:>10}   {fp:>10}")
else:
    print("  Disentanglement not performed.")

### Optimal delays

Neurons often respond to stimuli with a lag. Optimal delay estimation shifts
each feature to maximize MI, capturing delayed responses that fixed-lag
analysis would miss.

Temporal offset maximizing MI between neural activity and behavior.
Positive delays mean neural activity lags behind behavior (expected
for calcium imaging due to indicator dynamics).

In [ ]:
optimal_delays = info3.get('optimal_delays')
if optimal_delays is not None:
    fps = CONFIG['fps']
    neuron_types = ground_truth.get('neuron_types', {})
    type_delays = {}

    for neuron_id, features in significant_neurons3.items():
        ntype = neuron_types.get(neuron_id, 'unknown')
        for feat_name in features:
            if feat_name in feat_bunch:
                fi = feat_bunch.index(feat_name)
                d = optimal_delays[neuron_id, fi]
                type_delays.setdefault(ntype, []).append(d / fps)

    # Mean delay per neuron type
    print('Mean optimal delay per neuron type:')
    for ntype in sorted(type_delays):
        vals = type_delays[ntype]
        print(f'  {ntype:20s}  {np.mean(vals):+.2f}s  (n={len(vals)})')
else:
    print('No delay optimization performed.')

In [ ]:
fig = plt.figure(figsize=(14, 10))

# 1. Selectivity heatmap (main plot)
ax1 = fig.add_subplot(2, 2, (1, 2))

feature_names = feat_bunch
n_neurons = exp3.n_cells
n_features = len(feature_names)

# Create MI matrix using 'me'
mi_matrix = np.zeros((n_neurons, n_features))
for neuron_id, features in significant_neurons3.items():
    for feat_name in features:
        if feat_name in feature_names:
            feat_idx = feature_names.index(feat_name)
            pair_stats = exp3.get_neuron_feature_pair_stats(neuron_id, feat_name)
            mi_matrix[neuron_id, feat_idx] = pair_stats.get("me", 0)

im = ax1.imshow(mi_matrix, aspect="auto", cmap="viridis")
ax1.set_xlabel("Features")
ax1.set_ylabel("Neurons")
ax1.set_title("INTENSE selectivity heatmap (MI values)")
ax1.set_xticks(range(n_features))
ax1.set_xticklabels(feature_names, rotation=45, ha="right")

# Add colorbar
cbar = plt.colorbar(im, ax=ax1, shrink=0.8)
cbar.set_label("Mutual information (bits)")

# Add neuron type annotations
type_colors = {
    "hd_cells": "red",
    "place_cells": "blue",
    "speed_cells": "green",
    "event_cells": "orange",
    "mixed_cells": "purple",
    "nonselective": "gray",
}
for neuron_id, neuron_type in ground_truth["neuron_types"].items():
    color = type_colors.get(neuron_type, "gray")
    ax1.scatter(-0.7, neuron_id, c=color, s=20, marker="s")

# 2. Detection rates by type
ax2 = fig.add_subplot(2, 2, 3)
types = list(metrics["type_stats"].keys())
sensitivities = [metrics["type_stats"][t]["sensitivity"] * 100 for t in types]
colors = [type_colors.get(t, "gray") for t in types]

bars = ax2.bar(range(len(types)), sensitivities, color=colors)
ax2.set_xticks(range(len(types)))
ax2.set_xticklabels([t.replace("_", "\n") for t in types], fontsize=8)
ax2.set_ylabel("Detection rate (%)")
ax2.set_title("Detection rate by neuron type")
ax2.set_ylim(0, 105)
ax2.axhline(y=100, color="k", linestyle="--", alpha=0.3)

# Add percentage labels
for bar, pct in zip(bars, sensitivities):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            f"{pct:.0f}%", ha="center", va="bottom", fontsize=9)

# 3. Summary statistics (before and after disentanglement)
ax3 = fig.add_subplot(2, 2, 4)
ax3.axis("off")

summary_text = (
    f"VALIDATION SUMMARY\n"
    f"{'=' * 30}\n\n"
    f"{'Metric':<12} {'Raw':>8}\n"
    f"{'-' * 30}\n"
    f"{'Sensitivity':<12} {metrics['sensitivity']:>7.1%}\n"
    f"{'Precision':<12} {metrics['precision']:>7.1%}\n"
    f"{'F1 Score':<12} {metrics['f1']:>7.1%}\n\n"
    f"Detection counts:\n"
    f"  True Positives:  {metrics['true_positives']}\n"
    f"  False Positives: {metrics['false_positives']}\n"
    f"  False Negatives: {metrics['false_negatives']}\n\n"
    f"Population:\n"
    f"  Neurons: {exp3.n_cells}, Features: {len(exp3.dynamic_features)}\n"
    f"  Expected pairs: {len(ground_truth['expected_pairs'])}\n"
)
ax3.text(0.05, 0.95, summary_text, transform=ax3.transAxes,
        fontfamily="monospace", fontsize=9, verticalalignment="top")

plt.tight_layout()
plt.show()

Once analysis is complete, save the results for later visualization or
cross-session comparison.

### Save and load results

Persist INTENSE results to disk with [`save_results`](https://driada.readthedocs.io/en/latest/api/intense/base.html#driada.intense.io.save_results) and reload them
with [`load_results`](https://driada.readthedocs.io/en/latest/api/intense/base.html#driada.intense.io.load_results) for later analysis.

In [ ]:
# Save/load round-trip
with tempfile.TemporaryDirectory() as tmpdir:
    results_path = os.path.join(tmpdir, "intense_results.npz")
    save_results(results3, results_path)
    file_mb = os.path.getsize(results_path) / 1024 / 1024
    print(f"  Saved results: {results_path} ({file_mb:.1f} MB)")

    loaded = load_results(results_path)
    print(f"  Reloaded: {len(loaded.stats)} neurons")
    print(f"  Stats keys match: {set(str(k) for k in results3.stats.keys()) == set(loaded.stats.keys())}")

## Further reading

Standalone examples (run directly, no external data needed):
- [full_intense_pipeline](https://github.com/iabs-neuro/driada/tree/main/examples/full_intense_pipeline) -- Complete pipeline across all feature types
- [mixed_selectivity](https://github.com/iabs-neuro/driada/tree/main/examples/mixed_selectivity) -- Disentanglement and interaction information
- [signal_association](https://github.com/iabs-neuro/driada/tree/main/examples/signal_association) -- MI estimators, TDMI, conditional MI
- [behavior_relations](https://github.com/iabs-neuro/driada/tree/main/examples/behavior_relations) -- Feature-feature significance testing

[All examples](https://github.com/iabs-neuro/driada/tree/main/examples)